### This file is run in the second step of the process.  It loads the data file created in step one and then generates the model.pkl file for predictions in the future. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import requests
import json
from pathlib import Path
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print("Pandas and Modeling dependencies loaded")

Pandas and Modeling dependencies loaded


In [2]:
data = Path('./results/DataProcessingExtractFile-RawData.csv')
df = pd.read_csv(data)
df.head(10)

,Diabetes_Status,HighBP,HighChol,CholCheck,BMI_Range,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,Mental_Health_Range,Physical_Health_Range,DiffWalk,Sex,Age,Education,Income
0,1,1.0,1.0,1.0,4,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,4,3,1.0,0.0,9.0,4.0,3.0
1,1,0.0,0.0,0.0,3,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,1,1,0.0,0.0,7.0,6.0,1.0
2,1,1.0,1.0,1.0,3,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,6,6,1.0,0.0,9.0,4.0,8.0
3,1,1.0,0.0,1.0,3,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,1,1,0.0,0.0,11.0,3.0,6.0
4,1,1.0,1.0,1.0,2,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,1,1,0.0,0.0,11.0,5.0,4.0
5,1,1.0,1.0,1.0,3,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,1,1,0.0,1.0,10.0,6.0,8.0
6,1,1.0,0.0,1.0,4,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,3.0,1,3,0.0,0.0,9.0,6.0,7.0
7,1,1.0,1.0,1.0,3,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,1,1,1.0,0.0,11.0,4.0,4.0
8,2,1.0,1.0,1.0,4,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,5.0,6,6,1.0,0.0,9.0,5.0,1.0
9,1,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,1,1,0.0,1.0,8.0,4.0,3.0


In [3]:
# Create Random Forest Model
X = df.copy()
X = df.drop(df.columns[[0]], axis=1)
X.head()

,HighBP,HighChol,CholCheck,BMI_Range,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,Mental_Health_Range,Physical_Health_Range,DiffWalk,Sex,Age,Education,Income
0,1.0,1.0,1.0,4,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,5.0,4,3,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,3,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,3.0,1,1,0.0,0.0,7.0,6.0,1.0
2,1.0,1.0,1.0,3,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,5.0,6,6,1.0,0.0,9.0,4.0,8.0
3,1.0,0.0,1.0,3,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,1,1,0.0,0.0,11.0,3.0,6.0
4,1.0,1.0,1.0,2,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,1,1,0.0,0.0,11.0,5.0,4.0


In [4]:
y = df["Diabetes_Status"].ravel()
y[:5]

array([1, 1, 1, 1, 1], dtype=int64)

In [5]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    random_state=78,
                                                    stratify=y)

In [6]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [7]:
# Fitting the model
rf_model = rf_model.fit(X_train, y_train)

In [8]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test)
predictions

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [9]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

In [10]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.8347682119205299

In [11]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,50677,2749
Actual 1,7730,2264


Accuracy Score : 0.8347682119205299
Classification Report
              precision    recall  f1-score   support

           1       0.87      0.95      0.91     53426
           2       0.45      0.23      0.30      9994

    accuracy                           0.83     63420
   macro avg       0.66      0.59      0.60     63420
weighted avg       0.80      0.83      0.81     63420



In [12]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.04701055, 0.0309589 , 0.00481236, 0.05461734, 0.04176785,
       0.01519345, 0.02102142, 0.03322141, 0.04021389, 0.03313663,
       0.00999505, 0.01128344, 0.01971693, 0.08608625, 0.04370586,
       0.05165071, 0.02926573, 0.03498013, 0.16685883, 0.08920929,
       0.13529399])

In [13]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.16685882500252905, 'Age'),
 (0.13529398562885284, 'Income'),
 (0.08920928862467696, 'Education'),
 (0.08608624992600782, 'GenHlth'),
 (0.054617335780717056, 'BMI_Range'),
 (0.05165071294875037, 'Physical_Health_Range'),
 (0.04701055155896745, 'HighBP'),
 (0.04370586203700365, 'Mental_Health_Range'),
 (0.041767854489508316, 'Smoker'),
 (0.04021388651737167, 'Fruits'),
 (0.03498012958032994, 'Sex'),
 (0.033221409973853595, 'PhysActivity'),
 (0.03313662993838445, 'Veggies'),
 (0.03095890418406595, 'HighChol'),
 (0.029265729769639055, 'DiffWalk'),
 (0.021021418207909787, 'HeartDiseaseorAttack'),
 (0.019716932164332963, 'NoDocbcCost'),
 (0.015193449284070341, 'Stroke'),
 (0.011283439416146585, 'AnyHealthcare'),
 (0.0099950483888361, 'HvyAlcoholConsump'),
 (0.004812356578046106, 'CholCheck')]

In [14]:
pickle.dump(rf_model, open('model.pkl','wb'))